<a href="https://colab.research.google.com/github/netmatze/mlmatze/blob/main/esrgan_to_increase_stable_diffusion_images_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --upgrade diffusers transformers accelerate mediapy triton scipy ftfy spacy==3.4.4
!pip install -q xformers==0.0.16rc425
!pip install mediapy

### increase image resolution

In [ ]:
#list_of_selected_images = [5,6,7,11]
#list_of_selected_images = [1,2,3,7,8]
#list_of_selected_images = [2,3,4,5]
list_of_selected_images = [6,7,13,14]

In [ ]:
# from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler, DPMSolverMultistepScheduler
# import mediapy as media
# import torch
# from diffusers import StableDiffusionPipeline
# import random
# from PIL import Image
# import matplotlib.pyplot as plt

combined_list_images = []
combined_list_generated_models = []
combined_list_prompts = []
combined_list_generated_tags = []

def null_safety(images, **kwargs):
    return images, False

def flatten(listItems):
    return [item for sublist in listItems for item in sublist]

model_ids = [
              "stabilityai/stable-diffusion-2-1",
              # "runwayml/stable-diffusion-v1-5",
              #"dreamlike-art/dreamlike-photoreal-2.0",
              "dreamlike-art/dreamlike-anime-1.0",
              "dreamlike-art/dreamlike-diffusion-1.0",
              # "hakurei/waifu-diffusion",
              # "andite/anything-v4.0",
              #"ckpt/anything-v4.5",
              "Joeythemonster/anything-midjourney-v-4-1",
              #"prompthero/openjourney",
              # "andite/cutesexyrobutts-diffusion",
              #"hassanblend/HassanBlend1.5.1.2",
              #"OfficialUnstableDiffusion/UnstablePhotoRealv.5",
              #"ImNoOne/f222-inpainting-diffusers",
              #"SY573M404/f222-diffusers"
            ]
model_index = [False, False, True, True, True, True, True, True]
device = "cuda"

num_images = 4
#seed = random.randint(0, 2147483647)

#prompt = "girl, aqua eyes, yellow baseball cap, with a cute cat"
#prompt = "best quality, girl, blonde hair, pink eyes with a cute little dog"
#prompt = "masterpiece, best quality, sunnflowers and orange roses at the moon"
#prompt = "james rizzi style easter bunny rides with bike to the moon to hide easter eggs"
#prompt = "best quality, a pattern of different meadow flowers and daisy"
#prompt = "best quality, a cat with a hat riding a horse"

ideas = [
      "timeseries, prediction, stocks, machine learning, model arima, deep learning, high detail, neon design, intricate detail",
      "regularization, explaination, ridge, lasso, regression, high detail, intricate detail, cinematic lighting, metal design",
      #"sugar cube, cup of tea, sugar, diffusing, high detail, realistic, intricate detail",
      #"person, walking, misty forest, trees, plants, high detail, realistic, intricate detail",
      #"drop of oil on water, oil diffusing, colorful pattern, high detail, realistic, intricate detail",
      # "woman, knitting, wool, outside, colorfull pattern, high detail, realistic, intricate detail",
      # "strumming a guitar, sound, air, diffusing, colorful pattern, high detail, realistic, intricate detail",
      # "bubbles in a park, bubbles diffusing, floating away, colorful pattern, high detail, realistic, intricate detail",
#      "laptop, VPN with a lock symbol, high detail, realistic, intricate detail,",
#      "firewall, block unauthorized access, network, high detail, realistic, intricate detail",
#     "young asian woman, highly detailed, green eyes, bathing in wool, knitting a pullover, realistic",
#     "Anthropomorphic blue owl, highly detailed, big green eyes, portrait, detailed armor, unreal engine, cinematic lighting, metal design, 8k, octane render, realistic, redshift render",
#     "enraged warrior, monsterlike armor, living armor, full body portrait, organic armor, high detail, intricate detail, mdjrny-v4 style",
#     "young black woman, nude, big breasts, green eyes, high detail, intricate detail",
#      "polar bear, flying in space, moon in the background, sun on the right side with bright red glow, everything neon light like",
#      "polar bear, padlock and key, binary code background, network, high detail, realistic, intricate detail, mdjrny-v4 style",
#      "cat, virtual private network, VPN, smartphone, high detail, realistic, intricate detail",
#      "keyboard, python coding language in the background, high detail, realistic, intricate detail",
#      "blue owl, highly detailed, big green eyes, portrait, detailed armor, cinematic lighting, metal design, 8k, octane render, realistic, redshift render"
]

# ideas = [
#     # "young asian woman, nude, big breasts, green eyes, realistic, high detail, intricate detail, full body portrait",
#     # "nude blond girl, sitting at a couch, nude, open pussy, big breasts, high detail, realistic, intricate detail, full body portrait",
#     "nude red head girl, standing near a tree, nude, finger in open pussy, small breasts, high detail, realistic, intricate detail, full body portrait"
#     "nude man, fucking, nude girl, open pussy, big tits, high detail, realistic, intricate detail, "
# ]

tags = [
     ["woman", "cat", "blond hair"],
     ["whiteboard"],
     ["scanner"],
    # ["sugar cube", "cup of tea", "sugar", "diffusing"],
    # ["person", "walking", "misty forest", "trees", "plants"],
    # ["drop of oil on water", "oil diffusing", "colorful pattern"],
    # ["woman", "knitting", "wool", "colorful pattern"],
    # ["guitar", "air", "diffusing", "air"],
    # ["bubbles", "diffusing", "floating", "colorful pattern"],
    # ["chemistry", "science", "laboratory", "diffusion gradient", "flask"],
    # ["perfume", "cologne", "fragrance", "diffusion", "room"],
    # ["breath", "water vapor", "diffusion", "cold weather", "condensation"],
    # ["smoke", "incense", "candle", "diffusion", "atmosphere"],
    # ["cooking", "soup", "aroma", "diffusion", "kitchen"],
    # ["soap bubbles", "diffusion", "pattern", "beauty", "fun"],
    # ["cityscape", "smog", "fog", "diffusion", "atmosphere"],
    # ["air freshener", "scent", "diffusion", "room", "fresh"],
    # ["flower", "scent", "aroma", "diffusion", "nature"],
    # ["dandelion", "seeds", "diffusion", "nature", "beauty"],
    # ["coffee", "milk", "mixing", "beverage", "breakfast"],
    # ["garden", "hose", "watering", "lawn", "irrigation"],
    # ["music", "speaker", "sound waves", "vibration", "entertainment"],
    # ["fan", "cooling", "air currents", "summer", "heat"],
    # ["dominoes", "falling", "sequence", "game", "entertainment"],
    # ["cooking", "soup", "stirring", "ingredients", "comfort food"],
    # ["wind chime", "breeze", "sound waves", "relaxation", "decoration"],
    # ["waterfall", "cascade", "rocks", "pool", "nature"],

    # ["dye", "water", "color", "diffusion", "chemistry"],
    # ["eyes", "iris", "dilation", "light", "vision"],
    # ["solar panel", "sunlight", "energy", "diffusion", "renewable"],
    # ["wax", "mold", "filling", "diffusion", "craft"],
    # ["face", "mist", "essential oils", "diffusion", "aromatherapy"],
    # ["smoke bomb", "smoke", "colorful", "diffusion", "outdoors"],
    # ["city street", "crowded", "traffic", "diffusion", "urban"],
    # ["glitter", "sparkle", "airborne", "diffusion", "celebration"],
    # ["stress ball", "pressure", "relief", "diffusion", "handheld"],
    # ["ice cream cone", "melting", "dripping", "diffusion", "dessert"],
    # ["sugar cube", "tea", "sweetening", "diffusion", "beverage"],
    # ["ear", "sound waves", "hearing", "diffusion", "anatomy"],
    # ["forest", "misty", "atmosphere", "diffusion", "nature"],
    # ["oil", "water", "pattern", "diffusion", "art"],
    # ["guitar", "strumming", "music", "diffusion", "entertainment"],
    # ["bubbles", "floating", "playful", "diffusion", "childhood"],
    # ["tomato", "juices", "cutting board", "diffusion", "cooking"],
    # ["feet", "sandy beach", "walking", "diffusion", "vacation"]
    # ["network scanning", "vulnerability assessment", "penetration testing", "ethical hacking", "cybersecurity"],
    # ["firewall", "network security", "data protection", "information security", "internet security"],
    # ["animal", "bear", "polar bear", "snow", "moon", "neon"],
    # ["animal", "bear", "polar bear", "padlock", "key", "neon"],
    # ["animal", "cat", "virtual private network", "VPN", "smartphone"],
    # ["keyboard", "python", "coding", "language"],
    # ["animal", "owl", "portrait", "green eyes"],
    ]

In [ ]:
#list_of_selected_images = [5,6,7,11,14,16,18,19]
#list_of_selected_images = [1,2,3,5,13,14]
#list_of_selected_images = [1,2,3,7,8]
#list_of_selected_images = [2,3,4,5,6,7,13,14]
#list_of_selected_images = [2,3,4,5]
list_of_selected_images = [15]

In [ ]:
for index, promptvalue in enumerate(ideas):
  prompt = promptvalue
  tag = tags[index]
  list_images = []
  list_generated_models = []
  num_images = 2
  for index, model_id in enumerate(model_ids):
    for number_images in range(num_images):
      combined_list_generated_models.append(model_id)
      combined_list_prompts.append(prompt)
      combined_list_generated_tags.append(tag)

print(combined_list_generated_models)
print(combined_list_prompts)
print(combined_list_generated_tags)

['stabilityai/stable-diffusion-2-1', 'stabilityai/stable-diffusion-2-1', 'dreamlike-art/dreamlike-anime-1.0', 'dreamlike-art/dreamlike-anime-1.0', 'dreamlike-art/dreamlike-diffusion-1.0', 'dreamlike-art/dreamlike-diffusion-1.0', 'Joeythemonster/anything-midjourney-v-4-1', 'Joeythemonster/anything-midjourney-v-4-1', 'stabilityai/stable-diffusion-2-1', 'stabilityai/stable-diffusion-2-1', 'dreamlike-art/dreamlike-anime-1.0', 'dreamlike-art/dreamlike-anime-1.0', 'dreamlike-art/dreamlike-diffusion-1.0', 'dreamlike-art/dreamlike-diffusion-1.0', 'Joeythemonster/anything-midjourney-v-4-1', 'Joeythemonster/anything-midjourney-v-4-1', 'stabilityai/stable-diffusion-2-1', 'stabilityai/stable-diffusion-2-1', 'dreamlike-art/dreamlike-anime-1.0', 'dreamlike-art/dreamlike-anime-1.0', 'dreamlike-art/dreamlike-diffusion-1.0', 'dreamlike-art/dreamlike-diffusion-1.0', 'Joeythemonster/anything-midjourney-v-4-1', 'Joeythemonster/anything-midjourney-v-4-1', 'stabilityai/stable-diffusion-2-1', 'stabilityai/st

In [ ]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(SAVED_MODEL_PATH)

def preprocess_image(image_path):
  """ Loads image from path and preprocesses to make it model ready
      Args:
        image_path: Path to the image file
  """
  hr_image = tf.image.decode_image(tf.io.read_file(image_path))
  # If PNG, remove the alpha channel. The model only supports
  # images with 3 color channels.
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)

def generate_image(image):
  """
    Saves unscaled Tensor Images.
    Args:
      image: 3D image tensor. [height, width, channels]
      filename: Name of the file to save.
  """
  if not isinstance(image, Image.Image):
    image = tf.clip_by_value(image, 0, 255)
    image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
    print("Generated Image .jpg")
  return image

%matplotlib inline
def plot_image(image, title=""):
  """
    Plots images from image tensors.
    Args:
      image: 3D image tensor. [height, width, channels].
      title: Title to display in the plot.
  """
  image = np.asarray(image)
  image = tf.clip_by_value(image, 0, 255)
  image = Image.fromarray(tf.cast(image, tf.uint8).numpy())
  plt.imshow(image)
  plt.axis("off")
  plt.title(title)

from google.colab import files

for i in list_of_selected_images:
  selected_image_path = f"{i}.png"
  selected_image = preprocess_image(selected_image_path)
  print(selected_image.shape)
  # plot_image(tf.squeeze(selected_image), title=f"Original Image - {i}.png")
  start = time.time()
  print(type(selected_image))
  selected_super_image = model(selected_image)
  selected_super_image = tf.squeeze(selected_super_image)
  print('Time taken to complete process: %f'%(time.time() - start))
  # plot_image(tf.squeeze(selected_super_image),f'Super Resolution - {i}.png')

  prompt = combined_list_prompts[i - 1]
  model_id = combined_list_generated_models[i - 1]
  tag_values = combined_list_generated_tags[i - 1]

  download_image = generate_image(selected_super_image)
  download_image.save(f"{i}_{prompt}_{tag_values}.jpg")
  files.download(f'{i}_{prompt}_{tag_values}.jpg')

(1, 768, 768, 3)
<class 'tensorflow.python.framework.ops.EagerTensor'>
Time taken to complete process: 169.478286
Generated Image .jpg


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>